1.Import all libraries and scraping one
2. Import all libraries we used for SQL Lite
3. Query our database and retrieve all entries
4. For each entry we'll scrape the listing date, listing price from URL that's in database
5. Update database

In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3
conn = sqlite3.connect('sanfrancisco.db')
import datetime
import time

In [20]:
!pip install splinter
from splinter import Browser
from bs4 import BeautifulSoup
!which chromedriver
import time

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/usr/local/bin/chromedriver


In [30]:
executable_path = {'executable_path' : '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [50]:
def scrape(url):

#     url = 'https://www.redfin.com/city/17151/CA/San-Francisco/filter/property-type=house,min-beds=3,min-baths=2,include=sold-3mo'
    browser.visit(url)
    time.sleep(2)
    
    html=browser.html
    soup=BeautifulSoup(html, 'html.parser')
    
    
       headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"}
       response = requests.get(url, headers=headers)



    soup = BeautifulSoup(response.text,'html.parser')
    results = soup.find_all('span', class_="entryItemContent")
    

    original_price=None
    for result in results:
        try:
            index = result.text.find("Original Price: ")
            if index != -1:
                price=result.text[16:len(result.text)]
                print(price)
                original_price=int(float(price.replace(',','').replace('$','')))
                print("Original Price: " + (str(original_price)))
                  
                break
      
        except AttributeError as e:
            print(e)
            
    print(original_price)
    
    listing_date=None
    for result in results:
        try:
            search_string = "On Market Date: "
            index = result.text.find(search_string)
            if index != -1:
                listing_date=result.text[len(search_string):len(result.text)]
                print("Listing Date: " + (str(listing_date)))
                listing_date=int(datetime.datetime.strptime(listing_date,"%A, %B %d, %Y").strftime("%s"))

                break

        except AttributeError as e:
                print(e)
                print(listing_date)
    return original_price, listing_date

In [67]:
c = conn.cursor()
c.execute("SELECT Address, URL FROM Address WHERE Original_Price IS Null")

rows = c.fetchall()
iteration = 1

missing_data = [
    "2924 Rivera St",
    "1230 5th Ave",
    "87 Melrose Ave",
    "2211 44th Ave",
    "860 De Haro St",
    "701 Quintara St",
    "345 Holly Park Cir",
    "160 Anderson St",
    "156 Midcrest Way",
    "51 Somerset St",
    "2038 20th Ave",
    "1727 31st Ave",
    "2271 40th Ave",
    "525 Holloway Ave",
    "34 Wanda St"
]

for row in rows:
    address = row[0]
    
    # Missing data on web page
    if address in missing_data:
        continue
    
    url = row[1]
    print(address)
    print(url)
    print(row)
    
    redfin=scrape(url)
    original_price = redfin[0]
    listing_date = redfin[1]
    
    c.execute('''UPDATE Address SET Original_Price = ? , Listing_date = ? WHERE Address = ?
            ''', (original_price,listing_date,address))
    conn.commit()

    print("Completed " + str(iteration) + " of " + str(len(rows)) + "...")
    iteration += 1
    
    if iteration > 10:
        print("stopping iterations...")
        break
    
    time.sleep(15)


In [14]:
#Parse the string to add the Listing Price/Sold Price
# Talk about the bot detection of Redfin